In [1]:
import logging
import os
import re
from urllib.parse import urlparse, parse_qs
import tempfile


from bs4 import BeautifulSoup
import jinja2
import requests

from tessa_chef import make_request, get_parsed_html_from_url, make_fully_qualified_url
from tessa_chef import get_license, get_text
from tessa_chef import get_section_filename, create_predictable_zip, download_file
from tessa_chef import shutil



from le_utils.constants import content_kinds, file_formats, licenses

import pprint
pp = pprint.PrettyPrinter(indent=4, width=80)


## Test download_module funcion


In [2]:
# from tessa_chef import download_module
# test_module = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81108' # EN
# test_module = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82912' # AR
test_module = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82514' # FR

# toc = download_module(test_module)


In [3]:
# prints TOC
# for section in toc['children']:
#     print('  - section:', section['title'], section['filename'])
#     for subsection in section['children']:
#         print('    - subsection:', subsection['title'], subsection['filename'])




## Test scraping


In [3]:
from tessa_chef import TessaChef
tessa_chef = TessaChef()
tessa_chef.scrape(None, {'lang':'fr'})

Scrapring module @ url = http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=105334
destination= /var/folders/k3/r74jr38d56v717n39fd073f80000gn/T/tmpdrsqxcwf


AttributeError: 'NoneType' object has no attribute 'find_parent'

In [2]:

from tessa_chef import urlparse, parse_qs, get_parsed_html_from_url
from tessa_chef import _get_next_section_url, download_page, download_module_no_toc


test_module = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=82514' # FR
download_module_no_toc(test_module, lang='fr')

destination= /var/folders/k3/r74jr38d56v717n39fd073f80000gn/T/tmpihwglq_b
Scrapring section/subsectino... section-1.html
  - section: Module 1 : Étude du nombre et de la structure
Scrapring section/subsectino... section-1_1.html
    - subsection: Module 1 : Étude du nombre et de la structure: 1. Utilisation de jeux pour développer le calcul mental
Scrapring section/subsectino... section-1_2.html
    - subsection: Module 1 : Étude du nombre et de la structure: 2. Identifier les mathématiques pratiquées dans le jeu – Jouer en groupes
Scrapring section/subsectino... section-1_3.html
    - subsection: Module 1 : Étude du nombre et de la structure: 3. Utiliser des jeux culturels traditionnels
Scrapring section/subsectino... section-1_4.html
    - subsection: Module 1 : Étude du nombre et de la structure: Ressource 1: Jeux d’associations de nombres
Scrapring section/subsectino... section-1_5.html
    - subsection: Module 1 : Étude du nombre et de la structure: Ressource 2: Jeux utilisant des

'/var/folders/k3/r74jr38d56v717n39fd073f80000gn/T/tmpxkkfwwka.zip'

In [2]:
import os
os.getcwd()

'/Users/ivan/Projects/FLECode/sushi-chef-tessa/notebooks'

In [ ]:
os.path.

In [2]:
from tessa_chef import get_parsed_html_from_url
url = 'http://www.open.edu/openlearncreate/mod/oucontent/view.php?id=81337'
doc = get_parsed_html_from_url(url)


In [5]:
current_li_deep = doc.find('li', class_='oucontent-tree-current')
current_li_deep

module_toc_li = current_li_deep.find_parent('li', class_='item-section')
# print(module_toc_li.prettify())
# module_contents_div = module_toc_li.find('div', class_='oucontent-contents')
outer_module_ul = module_toc_li.find('ul', class_='child-item-list', recursive=False)
inner_module_ul = outer_module_ul.find('div', class_='oucontent-contents').find('ul', recursive=False)

section_lis = inner_module_ul.find_all('li', recursive=False)
print(len(section_lis))

16


In [11]:
for section_li in section_lis:
    section_title_span = section_li.find('span', class_='oucontent-tree-item')
    print(section_title_span.get_text())
    subsections_ul = section_li.find('ul', recursive=False)
    if subsections_ul:
        subsection_lis = subsections_ul.find_all('li')
        print(len(subsection_lis))
        for subsection_li in subsection_lis:
            subsection_link = subsection_li.find('a')
            if subsection_link:
                linkexists = 'linkexists'
            else:
                linkexists = 'NOLINK <<<<<<<<<<<<<<<'
            print('    ', subsection_li.get_text(), linkexists)
            #subsection_link = subsection_li.find('a')
            #print('   ', subsection_link.get_text())



TESSA Inclusive Education Toolkit
2
     A guide to the education and training of teachers in inclusive education NOLINK <<<<<<<<<<<<<<<
     Welcome to the Inclusive Education Toolkit! linkexists
1. Different types of schools
2. The inclusive teacher’s attitude and behaviour
8
     2.1 An inclusive teacher respects the individuality of each child linkexists
     2.2 Using appropriate words when talking about disabled children and their disability linkexists
     2.3 An inclusive teacher is aware of the different needs of pupils linkexists
     2.4 An inclusive teacher helps pupils to gain self-confidence and self-esteem linkexists
     2.5 An inclusive teacher helps pupils to feel included in the learning community linkexists
     2.6 An inclusive teacher acknowledges the contribution of each child linkexists
     2.7 An inclusive teacher contributes to the emotional well-being of all children linkexists
     2.8 Reflection on the inclusive teacher linkexists
3. A classroom for all in

In [13]:
from tessa_chef import get_parsed_html_from_url
u = 'http://www.open.edu/openlearncreate/mod/resource/view.php?id=81325'
rsrc = get_parsed_html_from_url(u)



NOT CACHED:http://www.open.edu/openlearncreate/mod/resource/view.php?id=81325


In [14]:
course_content_div = rsrc.find(class_="pagecontent-content")

In [17]:
course_content_div

<div class="pagecontent-content"><div><span id="maincontent"></span><h2>تنزيل الموارد الصوتية</h2><div class="resourcecontent"><div class="text_to_html"><span class="oumediafilter"><a class="oumedialinknoscript" href="http://www.open.edu/openlearncreate/pluginfile.php/173006/mod_resource/content/1/04_cast_the_spell%20(1).mp3?forcedownload=1">Download this audio clip.</a><span id="filter_mp3_14184098821"></span></span><script type="text/javascript">
//<![CDATA[
document.getElementById("filter_mp3_14184098821").innerHTML = "<iframe tabindex=\"0\" title=\"Audio player: 04_cast_the_spell%20(1).mp3\" width=\"342\" height=\"30\" frameborder=\"0\" scrolling=\"no\" style=\"overflow:hidden\" src=\"http:\/\/www.open.edu\/openlearncreate\/local\/mediahack\/v.php?title=Audio+player%3A+04_cast_the_spell%2520%281%29.mp3&amp;media_url=http%3A%2F%2Fwww.open.edu%2Fopenlearncreate%2Fpluginfile.php%2F173006%2Fmod_resource%2Fcontent%2F1%2F04_cast_the_spell%2520%281%29.mp3&amp;width=342&amp;height=30&amp;r

In [20]:
url = "http://www.open.edu/openlearncreate/pluginfile.php/157707/mod_resource/content/1/excursion_transcript.pdf"
import requests
r = requests.head(url)

In [24]:
for k,v in r.headers.items():
    print(k+':',v)

Date: Fri, 24 Nov 2017 01:07:35 GMT
Server: Apache
Set-Cookie: MoodleSessionworks=3e118ftaq1bcselrgdbn20gde0; path=/, MoodleSessionworks=h6sg87b5hohcafulbnlldff771; path=/
Expires: Fri, 24 Nov 2017 07:07:36 GMT
Cache-Control: public, max-age=21600, no-transform
Pragma: 
Content-Disposition: inline; filename="excursion_transcript.pdf"
Last-Modified: Tue, 30 Jun 2015 09:06:37 GMT
Accept-Ranges: bytes
Content-Length: 80931
Cneonction: close
Content-Type: application/pdf


In [22]:
!pwd

/Users/ivan/Projects/FLECode/sushi-chef-tessa/notebooks


'Vietnam_Oral_History_Gene_Feher-5GyaJKWAi-s.mp4'

In [11]:
# [download] Destination: Vietnam_Oral_History_Gene_Feher-5GyaJKWAi-s.f136.mp4
video_data['title']

'Vietnam Oral History  Gene Feher'

In [33]:
from collections import Counter
cnt = Counter()
cnt['as'] = 1
cnt.update({'as':33, 'bas':3})
cnt
cnt.update({})
cnt['kind'] += 1
cnt

Counter({'as': 34, 'bas': 3, 'kind': 1})

In [34]:
str(cnt)

"Counter({'as': 34, 'bas': 3, 'kind': 1})"

In [41]:
from urllib.parse import urljoin
urljoin('http://site.com/page.html/', 'yahoocom')

'http://site.com/page.html/yahoocom'